In [1]:
# begin here

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [3]:
weather_data = pd.read_csv("data weather bigger.csv")
weather_data

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2015-01-01,6.3,-0.6,13.9,0.0,NaN,NaN,2.8,NaN,1019.2,NaN
1,2015-01-02,7.2,0.0,15.0,0.0,NaN,NaN,2.2,NaN,1019.2,NaN
2,2015-01-03,8.3,1.7,16.1,0.0,NaN,NaN,2.6,NaN,1022.5,NaN
3,2015-01-04,11.9,3.3,20.6,0.0,NaN,NaN,5.2,NaN,1024.9,NaN
4,2015-01-05,18.9,15.0,24.4,0.0,NaN,354.0,22.7,NaN,1022.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3283,2023-12-28,12.7,8.3,19.4,0.0,NaN,357.0,1.1,NaN,1020.6,NaN
3284,2023-12-29,13.1,8.9,19.4,0.0,NaN,29.0,3.6,NaN,1018.8,NaN
3285,2023-12-30,13.0,8.9,16.7,14.8,NaN,354.0,5.5,NaN,1017.9,NaN
3286,2023-12-31,11.2,6.5,14.4,0.0,NaN,23.0,3.7,NaN,1018.8,NaN


In [4]:
weather_data = weather_data.drop(columns=['snow', 'wpgt', 'tsun', 'wdir'])

In [5]:
weather_data.to_csv('data weather bigger2.csv', index=False)

In [6]:
weather_data = pd.read_csv("data weather bigger2.csv")
weather_data.head(5)

,date,tavg,tmin,tmax,prcp,wspd,pres
0,2015-01-01,6.3,-0.6,13.9,0.0,2.8,1019.2
1,2015-01-02,7.2,0.0,15.0,0.0,2.2,1019.2
2,2015-01-03,8.3,1.7,16.1,0.0,2.6,1022.5
3,2015-01-04,11.9,3.3,20.6,0.0,5.2,1024.9
4,2015-01-05,18.9,15.0,24.4,0.0,22.7,1022.0


In [7]:
index = 1
weather_data.iloc[index:index+31].head(10)

,date,tavg,tmin,tmax,prcp,wspd,pres
1,2015-01-02,7.2,0.0,15.0,0.0,2.2,1019.2
2,2015-01-03,8.3,1.7,16.1,0.0,2.6,1022.5
3,2015-01-04,11.9,3.3,20.6,0.0,5.2,1024.9
4,2015-01-05,18.9,15.0,24.4,0.0,22.7,1022.0
5,2015-01-06,17.6,9.4,27.2,0.0,9.3,1021.1
6,2015-01-07,17.9,8.3,28.3,0.0,4.8,1018.9
7,2015-01-08,15.4,9.4,23.3,0.0,3.6,1016.5
8,2015-01-09,15.3,11.7,21.1,0.0,4.7,1015.4
9,2015-01-10,12.5,10.6,15.0,NaN,2.0,NaN
10,2015-01-11,14.2,12.2,16.7,9.3,4.7,NaN


In [8]:
df = weather_data.copy()
def fill_nan_with_avg_of_previous_3(column):
    # Convert column to a numpy array for efficient processing
    col_array = column.values
    # Iterate through the array to find NaNs
    for i in range(len(col_array)):
        if pd.isna(col_array[i]):
            # Get the preceding 3 values
            preceding_values = col_array[max(0, i-3):i]
            # Compute the average of the preceding values, if any
            if len(preceding_values) > 0:
                average = np.nanmean(preceding_values)
                col_array[i] = average
    return pd.Series(col_array)

# apply to each column
for col in df.columns:
    if df[col].isna().any():
        df[col] = fill_nan_with_avg_of_previous_3(df[col])


In [9]:
df.iloc[index:index+31].head(10)
df.to_csv('data weather bigger3.csv', index=False)

In [10]:
weather_data = pd.read_csv("data weather bigger3.csv")
weather_data.head(5)

,date,tavg,tmin,tmax,prcp,wspd,pres
0,2015-01-01,6.3,-0.6,13.9,0.0,2.8,1019.2
1,2015-01-02,7.2,0.0,15.0,0.0,2.2,1019.2
2,2015-01-03,8.3,1.7,16.1,0.0,2.6,1022.5
3,2015-01-04,11.9,3.3,20.6,0.0,5.2,1024.9
4,2015-01-05,18.9,15.0,24.4,0.0,22.7,1022.0


In [11]:
weather_data.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
wspd    float64
pres    float64
dtype: object

In [12]:
weather_data['date'] = pd.to_datetime(weather_data['date'])
weather_data['year'] = weather_data['date'].dt.year.astype('float64')
weather_data['month'] = weather_data['date'].dt.month.astype('float64')
weather_data['day'] = weather_data['date'].dt.day.astype('float64')
weather_data['day_of_week'] = weather_data['date'].dt.dayofweek.astype('float64') # 0=Monday, 6=Sunday
weather_data = weather_data.drop(columns=['date'])

In [13]:
weather_data.dtypes

tavg           float64
tmin           float64
tmax           float64
prcp           float64
wspd           float64
pres           float64
year           float64
month          float64
day            float64
day_of_week    float64
dtype: object

In [14]:
weather_data = weather_data.drop(index=424)
weather_data = weather_data.drop(index=1885)
weather_data = weather_data.drop(index=3287)
weather_data.reset_index(drop=True, inplace=True)

In [15]:
index = 421
weather_data.iloc[index:index+31].head(6)

,tavg,tmin,tmax,prcp,wspd,pres,year,month,day,day_of_week
421,18.5,10.0,27.2,0.0,3.6,1017.6,2016.0,2.0,26.0,4.0
422,18.2,11.7,26.7,0.0,6.1,1015.5,2016.0,2.0,27.0,5.0
423,19.7,12.8,27.2,0.0,8.5,1014.9,2016.0,2.0,28.0,6.0
424,19.2,11.7,28.3,0.0,4.4,1017.1,2016.0,3.0,1.0,1.0
425,17.8,11.1,25.0,0.0,5.3,1017.2,2016.0,3.0,2.0,2.0
426,16.4,11.1,23.9,0.0,3.8,1017.0,2016.0,3.0,3.0,3.0


In [16]:
index = 0*365
weather_data.iloc[index:index+31].head(6)

,tavg,tmin,tmax,prcp,wspd,pres,year,month,day,day_of_week
0,6.3,-0.6,13.9,0.0,2.8,1019.2,2015.0,1.0,1.0,3.0
1,7.2,0.0,15.0,0.0,2.2,1019.2,2015.0,1.0,2.0,4.0
2,8.3,1.7,16.1,0.0,2.6,1022.5,2015.0,1.0,3.0,5.0
3,11.9,3.3,20.6,0.0,5.2,1024.9,2015.0,1.0,4.0,6.0
4,18.9,15.0,24.4,0.0,22.7,1022.0,2015.0,1.0,5.0,0.0
5,17.6,9.4,27.2,0.0,9.3,1021.1,2015.0,1.0,6.0,1.0


In [17]:
X = pd.DataFrame()
y = pd.DataFrame()

for j in range(1,4):
    current_year = weather_data['year'].max()
    recent_years = [current_year - i for i in range(j,j+6)]
    filtered_weather_data_recent = weather_data[weather_data['year'].isin(recent_years)]
    #print(filtered_weather_data_recent)
    filtered_weather_data_current = weather_data[weather_data['year'].isin([current_year])]
    #print(filtered_weather_data_current)
    
    months_current = filtered_weather_data_current['month'].unique().astype(int)
    
    results_current = pd.DataFrame()
    
    # Start the loop for the current year's data or Y
    for month in months_current:
        max_day = int(filtered_weather_data_current[filtered_weather_data_current['month'] == month]['day'].max())
        
        for day in range(1, max_day + 1):
            # Filter the data for the current month and day
            grouped_current = filtered_weather_data_current[(filtered_weather_data_current['month'] == month) & (filtered_weather_data_current['day'] == day)]
            
            # Rename the 'tavg' column for the specific day
            grouped_tavg_current = grouped_current[['tavg']].rename(columns={'tavg': f'tavg_{month}_{day}'}).reset_index(drop=True)
            
            # Append results horizontally (axis=1)
            if not grouped_current.empty:
                results_current = pd.concat([results_current, grouped_tavg_current], axis=1)
    
    # Reset the index and transpose the DataFrame
    results_current = results_current.reset_index(drop=True).T
    print(results_current)
    y=pd.concat([y,results_current])
    
    
    # our X
    months = filtered_weather_data_recent['month'].unique().astype(int)
    
    results = pd.DataFrame()
    
    #start the loop
    for month in months:
        max_day = int(filtered_weather_data_recent[filtered_weather_data_recent['month'] == month]['day'].max())
        
        
        for day in range(1, max_day +1):
            #filter
            grouped = filtered_weather_data_recent[(filtered_weather_data_recent['month']==month) & (filtered_weather_data_recent['day'] ==day)]
            #grouped_tavg = grouped['tavg'].reset_index(drop=True)
            grouped_tavg = grouped[['tavg']].rename(columns={'tavg': f'tavg_{month}_{day}'}).reset_index(drop=True)
        
            
            #append results
            if not grouped.empty:
               results = pd.concat([results, grouped_tavg], axis=1)
    
    #reset index
    results = results.reset_index(drop=True)
    results = results.T
    print(results)
    X= pd.concat([X,results])


print("------------------------------------------------")
print(y)
print("------------------------------------------------")
print(X)

               0
tavg_1_1    12.7
tavg_1_2    10.9
tavg_1_3    12.3
tavg_1_4    14.5
tavg_1_5    13.6
...          ...
tavg_12_27  13.7
tavg_12_28  12.7
tavg_12_29  13.1
tavg_12_30  13.0
tavg_12_31  11.2

[365 rows x 1 columns]
               0     1     2     3     4     5
tavg_1_1     9.6  14.0  11.2  13.3  15.1  11.2
tavg_1_2    10.4  17.6  12.6  16.3  12.3  11.4
tavg_1_3    10.4  16.4  11.8  14.5  11.9   9.7
tavg_1_4    11.8  16.9  11.6  13.1  12.8  10.6
tavg_1_5    13.3  17.1  11.0  13.7  13.3  18.2
...          ...   ...   ...   ...   ...   ...
tavg_12_27  15.2  11.5  10.9  13.9   7.9  14.4
tavg_12_28  16.9  11.8   9.2   8.9   8.9  13.4
tavg_12_29  17.5  12.8   9.4  11.6   8.9  13.1
tavg_12_30  14.9  11.9  12.2  16.0  10.7  13.9
tavg_12_31  13.5  11.5  16.1  12.4  11.0  14.3

[365 rows x 6 columns]
               0
tavg_1_1    12.7
tavg_1_2    10.9
tavg_1_3    12.3
tavg_1_4    14.5
tavg_1_5    13.6
...          ...
tavg_12_27  13.7
tavg_12_28  12.7
tavg_12_29  13.1
tavg_12_30  13

In [18]:
#grouped = filtered_weather_data_recent[(filtered_weather_data_recent['month'] == 12) & (filtered_weather_data_recent['day'] == 30)]
#grouped_tavg = grouped['tavg'].reset_index(drop=True)
#grouped_tavg

In [19]:
#X = weather_data.drop('tavg', axis= 1)
#X = results
#y = results_current
#y = y.values.flatten()
#y = weather_data['tavg']
print(X.shape)
print(y.shape)

(1095, 6)
(1095, 1)


In [20]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(876, 6) (219, 6) (876, 1) (219, 1)


In [21]:
#define our model
model = Sequential()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
#layers
model.add(Input(shape=(X_train.shape[1],))) #input
model.add(Dense(150, activation='relu',kernel_regularizer=l2(0.0001)))   #hidden
model.add(BatchNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.001)))   #hidden
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu')) #output

In [23]:
#Compile
from tensorflow.keras.losses import Huber

model.compile(optimizer=Adam(),
              loss = Huber(),
              metrics=['accuracy'])

In [24]:
#training
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 100 == 0:  #freq
            print(f"Epoch {epoch}, Loss: {logs['loss']}, Val Loss: {logs['val_loss']}")

#training
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2, verbose=0, callbacks=[CustomCallback()])

Epoch 0, Loss: 16.423809051513672, Val Loss: 17.49793815612793
Epoch 100, Loss: 2.3141372203826904, Val Loss: 1.8457832336425781
Epoch 200, Loss: 1.9726362228393555, Val Loss: 1.815116047859192
Epoch 300, Loss: 2.025730609893799, Val Loss: 1.7876975536346436
Epoch 400, Loss: 2.002434492111206, Val Loss: 1.9474509954452515
Epoch 500, Loss: 1.826267957687378, Val Loss: 1.8158066272735596
Epoch 600, Loss: 1.8684794902801514, Val Loss: 1.81132173538208
Epoch 700, Loss: 1.8672879934310913, Val Loss: 1.8464747667312622
Epoch 800, Loss: 1.8225798606872559, Val Loss: 1.793704867362976
Epoch 900, Loss: 1.7418028116226196, Val Loss: 1.8072186708450317


In [25]:
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 1.6931 
Test Loss: [1.632792592048645, 0.0]


In [26]:
current_year = 2023
recent_years = [current_year - i for i in range(1,7)]
filtered_weather_data_recent = weather_data[weather_data['year'].isin(recent_years)]
month, day = 8, 5
grouped = filtered_weather_data_recent[(filtered_weather_data_recent['month']==month) & (filtered_weather_data_recent['day'] ==day)]
grouped_tavg = grouped[['tavg']].rename(columns={'tavg': f'tavg{month}_{day}'}).reset_index(drop=True).T
grouped_tavg

,0,1,2,3,4,5
tavg8_5,24.4,27.8,27.4,21.9,27.2,27.0


In [27]:
predictions = model.predict(X_test[0:1])
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


array([[18.414608]], dtype=float32)

In [28]:
grouped_tavg= scaler.transform(grouped_tavg)
model(grouped_tavg)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[25.391785]], dtype=float32)>